In [ ]:
!pip install boto3 --upgrade

In [9]:
import boto3
import json
boto3.__version__

'1.34.31'

In [2]:
def get_iam_execution_role():
    client = boto3.client('iam')
    
    response = client.get_role(
    RoleName='SandboxServiceRole'
                                )
    role = response['Role']['Arn']
    print(role)
    return role

In [3]:
def get_account_id():

    # Create an STS client
    sts_client = boto3.client('sts')

    # Get the caller identity
    caller_identity = sts_client.get_caller_identity()

    # Extract the account ID from the caller identity
    account_id = caller_identity['Account']
    return account_id



In [4]:
def create_bucket_for_canvas():
    account_id = get_account_id()
    bucket_name = f'sagemaker-canvas-us-east-1-{account_id}'
    s3 = boto3.client('s3')
    try:
        # Check if the bucket exists
        s3.head_bucket(Bucket=bucket_name)
        print(f"Bucket '{bucket_name}' already exists.")
        return(bucket_name)
    except Exception as e:
        # If the bucket doesn't exist, create it
        if e.response['Error']['Code'] == '404':
            s3.create_bucket(Bucket=bucket_name)
            print(f"Bucket '{bucket_name}' created successfully.")
            return(bucket_name)
    else:
        raise  # Raise other errors


In [5]:
bucket=create_bucket_for_canvas()

Bucket 'sagemaker-canvas-us-east-1-590116326836' already exists.


In [6]:
def get_network_settings():
    ec2=boto3.client('ec2')
    vpcs = ec2.describe_vpcs()
    
    vpcid = vpcs['Vpcs'][0]['VpcId']
    
    print(vpcid)
    
    subnets = ec2.describe_subnets()
    lst_subnets=[]
    for subnet in subnets['Subnets']:
        #print(subnet)
        #print(subnet['SubnetId'])
        lst_subnets.append(subnet['SubnetId'])
    print(lst_subnets)
    
    sggroups = ec2.describe_security_groups()
    lst_sgs=[]
    for sg in sggroups['SecurityGroups']:
        if 'default' in sg['GroupName']:
            lst_sgs.append(sg['GroupId'])
    print(lst_sgs)
    
    return(vpcid, lst_subnets, lst_sgs)

In [7]:
def create_sm_domain():
    vpc,subnets,securitygroups = get_network_settings()
    bucket_name = create_bucket_for_canvas()
    smclient = boto3.client('sagemaker')
    response = smclient.create_domain(
        DomainName='new-studio-domain',
        AuthMode='IAM',
        DefaultUserSettings={
            'ExecutionRole': get_iam_execution_role(),
            'SecurityGroups': securitygroups,
            
            'CanvasAppSettings': {
                'TimeSeriesForecastingSettings': {
                    'Status': 'DISABLED',
                    #'AmazonForecastRoleArn': 'string'
                },
                'ModelRegisterSettings': {
                    'Status': 'ENABLED',
                    #'CrossAccountModelRegisterRoleArn': 'string'
                },
                'WorkspaceSettings': {
                    'S3ArtifactPath': f's3://{bucket_name}',
                    #'S3KmsKeyId': 'string'
                },
    
            },
    
            'DefaultLandingUri': 'studio::',
            'StudioWebPortal': 'ENABLED',
    
        },
        DomainSettings={
            'SecurityGroupIds': securitygroups,
    
        },
        SubnetIds=subnets,
        VpcId=vpc,
    
        DefaultSpaceSettings={
            'ExecutionRole': get_iam_execution_role(),
            'SecurityGroups':securitygroups,
        }
    )
    return response

vpc-0b74cb8830dc55a22
['subnet-02c740e17fadafbcb', 'subnet-0175bb0a24cff7a46']
['sg-00114b3593c46996a', 'sg-0e2d4c9ad905c0a8a']
Bucket 'sagemaker-canvas-us-east-1-590116326836' already exists.
arn:aws:iam::590116326836:role/SandboxServiceRole
arn:aws:iam::590116326836:role/SandboxServiceRole


In [8]:
response

{'DomainArn': 'arn:aws:sagemaker:us-east-1:590116326836:domain/d-3dtwr6ytdu23',
 'Url': 'https://d-3dtwr6ytdu23.studio.us-east-1.sagemaker.aws',
 'ResponseMetadata': {'RequestId': '53fd4771-758f-4647-a4b7-94097074e6bb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '53fd4771-758f-4647-a4b7-94097074e6bb',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '140',
   'date': 'Wed, 31 Jan 2024 15:21:58 GMT'},
  'RetryAttempts': 0}}